In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from dask.distributed import Client, LocalCluster

In [2]:
# Criando o cluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

2022-07-28 17:46:38,392 - distributed.diskutils - INFO - Found stale lock file and directory '/home/joao/documents/Gabinete/Relatórios/Relatorio-Conectividade/Analise/Schema/dask-worker-space/worker-7nwnf6ni', purging
2022-07-28 17:46:38,392 - distributed.diskutils - INFO - Found stale lock file and directory '/home/joao/documents/Gabinete/Relatórios/Relatorio-Conectividade/Analise/Schema/dask-worker-space/worker-1jp5r9o_', purging
2022-07-28 17:46:38,393 - distributed.diskutils - INFO - Found stale lock file and directory '/home/joao/documents/Gabinete/Relatórios/Relatorio-Conectividade/Analise/Schema/dask-worker-space/worker-y3g7msbi', purging
2022-07-28 17:46:38,394 - distributed.diskutils - INFO - Found stale lock file and directory '/home/joao/documents/Gabinete/Relatórios/Relatorio-Conectividade/Analise/Schema/dask-worker-space/worker-oi8zrjz8', purging


In [3]:
pd.set_option('display.max_columns', None)

### Funções

In [4]:
def load_year_enem(year, cols):
    enem = dd.read_csv(
        f'../../Dados/Enem/microdados_enem_{year}/DADOS/MICRODADOS_ENEM_{year}.csv',
        usecols = cols,
        sep = ';',
        header = 0,
        #nrows=300*1000,
        encoding='latin-1',
        dtype={'NO_MUNICIPIO_ESC': 'object',
           'SG_UF_ESC': 'object',
           'CO_MUNICIPIO_ESC': 'object'},
        assume_missing=True
    )
    return enem

In [5]:
def filter_enem(df):
    pre_filter = len(df)
    print('Tamanho do dataset pré-filtragem: ', pre_filter)

    # Aplicando filtragem dos dados
    df = df[df['IN_TREINEIRO']==0] # Não é treineiro
    df = df[df['TP_ENSINO']==1] # Ensino Regular
    df = df[df['TP_ST_CONCLUSAO']!=4] # Está cursando ensino médio ou já cursou
    df = df[df['TP_ANO_CONCLUIU']<=1] # Está cursando ou concluiu o ensino médio até 2018
    df = df[df['TP_NACIONALIDADE']==1] # Brasileiro
    df = df[~df['CO_MUNICIPIO_ESC'].isnull()] # Possui identificador de município da escola associado

    # Removendo colunas pós filtragem
    df = df.drop(columns=['TP_ENSINO','IN_TREINEIRO','TP_ANO_CONCLUIU','TP_NACIONALIDADE','TP_ST_CONCLUSAO']) 

    post_filter = len(df)
    print('Tamanho do dataset pós-filtragem: ', post_filter)
    print('Taxa de filtragem: ', (pre_filter-post_filter)/pre_filter)
    return df

In [6]:
def agg_enem(df, index_cols, agg_funs, normalize=False):
    """
    Aggregate dataframe using specified columns and functions.
    """
    if 'percent' in agg_funs.keys():
        series = []
        for col in agg_funs['percent']:
            series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
        percented = pd.concat(series, axis=1)


    if 'median' and 'mean' in agg_funs.keys():
        del series
        print('Percentual calculado')
        median = df.compute().groupby(index_cols).agg(agg_funs['median']).add_prefix('median_')
        print('Mediana calculada')
        merged_median = pd.concat([median, percented], axis=1)
    
        mean = df.compute().groupby(index_cols).agg(agg_funs['mean']).add_prefix('mean_')
        print('Média calculada')
        merged_mean = pd.concat([mean, percented], axis=1)

        return merged_median, merged_mean
        
    return percented
    

In [7]:
def persist_df(df, file_name):
    '''
    Persists a dataframe to disk.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe to be persisted.
    '''
    df.to_parquet('../../Resultados/Agg/ENEM/'+ file_name)

In [20]:
def load_and_agg_enem(index_col, filter_cols, categorical_cols, agg_funs, normalize=False):
    '''
    Loads and aggregates the dataframe for the specified year.
    '''
    dfs = []
    for year in range(2019,2022):
        enem = filter_enem(load_year_enem(year, filter_cols))[categorical_cols + index_col]
        print('Ano carregado.')
        dfs.append(agg_enem(enem, index_col, agg_funs, normalize=normalize))
        print('Cálculo efetuado.')
        if normalize:
            persist_df(dfs[-1], f'anual/{year}_norm_enem.parquet')
        else:
            persist_df(dfs[-1], f'anual/{year}_enem.parquet')
        
        print(f'Ano {year} concluído')
    return pd.concat(dfs)

# Introdução

Para a análise dos microdados do Enem, serão utilizadas duas estratégias distintas:

- Estratégia 1: 

    variáveis desagregadas com o maior nível de detalhamento possível, com inferência dos pesos de cada variável explicativa na nota individual do aluno e utilizando as informações já disponíveis nos microdados do enem.
- Estratégia 2: 

    variáveis agregadas à nível de município, com adição de variáveis escolares a partir de base de dados de escolas (censo escolar, PBLE, etc.). Com isso, a análise será em nível macro, e pode-se associar ao investimento governamental

## Estratégia 1

In [9]:
cols_dict = {
    'NU_ANO': int,
    'TP_COR_RACA':int,
    'TP_SEXO':'string',
    'TP_FAIXA_ETARIA':int,
    'TP_ESTADO_CIVIL':int,
    'TP_ST_CONCLUSAO':int,
    'TP_ANO_CONCLUIU':int,
    'TP_NACIONALIDADE':int,
    'TP_ENSINO':int,
    'TP_ESCOLA':int,
    'IN_TREINEIRO':int,
    'CO_MUNICIPIO_ESC':'Sparse[int]',
    'NO_MUNICIPIO_ESC':'string',
    'SG_UF_ESC':'string',
    'TP_DEPENDENCIA_ADM_ESC':int,
    'TP_LOCALIZACAO_ESC':int,
    'TP_SIT_FUNC_ESC':int,
    'Q001': 'object',
    'Q002': 'object',
    'Q003': 'object',
    'Q004': 'object',
    'Q005': 'object',
    'Q006': 'object',
    'Q007': 'object',
    'Q008': 'object',
    'Q009': 'object',
    'Q010': 'object',
    'Q012': 'object',
    'Q019': 'object',
    'Q022': 'object',
    'Q024': 'object',
    'Q025': 'object',
    'NU_NOTA_CN':'Sparse[float]',
    'NU_NOTA_CH':'Sparse[float]',
    'NU_NOTA_LC':'Sparse[float]',
    'NU_NOTA_MT':'Sparse[float]',
    'TP_STATUS_REDACAO':int,
    'NU_NOTA_COMP1':'Sparse[int]',
    'NU_NOTA_COMP2':'Sparse[int]',
    'NU_NOTA_COMP3':'Sparse[int]',
    'NU_NOTA_COMP4':'Sparse[int]',
    'NU_NOTA_COMP5':'Sparse[int]',
    'NU_NOTA_REDACAO':'Sparse[int]'
            }

In [10]:
# pd_enem = pd.read_csv(
#     '../../Dados/Enem/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv',
#     usecols = cols_dict.keys(),
#     sep = ';',
#     header = 0,
#     nrows=300*1000,
#     encoding='latin-1',
#     dtype={'NO_MUNICIPIO_ESC': 'object',
#        'SG_UF_ESC': 'object',
#        'CO_MUNICIPIO_ESC': 'object'}
# )

In [11]:
# # Using Dask to read the data
# enem = dd.read_csv(
#     '../../Dados/Enem/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv',
#     usecols = cols_dict.keys(),
#     sep = ';',
#     header = 0,
#     encoding='latin-1',
#     dtype={'NO_MUNICIPIO_ESC': 'object',
#        'SG_UF_ESC': 'object',
#        'CO_MUNICIPIO_ESC': 'object'}
# )

Totalizando colunas categóricas ano a ano

In [12]:
index_col = ['NU_ANO']

In [13]:
categorical_cols = ['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ESCOLA', 'SG_UF_ESC', 'TP_LOCALIZACAO_ESC', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
       'Q010', 'Q012', 'Q019', 'Q022', 'Q024', 'Q025']

In [14]:
agg_funs = {
    'percent': categorical_cols,
}

In [21]:
agg = load_and_agg_enem(index_col=index_col, filter_cols=cols_dict.keys(), categorical_cols=categorical_cols, agg_funs=agg_funs, normalize=True)

Tamanho do dataset pré-filtragem:  5095171
Tamanho do dataset pós-filtragem:  1033757
Taxa de filtragem:  0.7971104404543047
Ano carregado.


/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified

Cálculo efetuado.
Ano 2019 concluído
Tamanho do dataset pré-filtragem:  5783109
Tamanho do dataset pós-filtragem:  825524
Taxa de filtragem:  0.8572525608630237
Ano carregado.


/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified

Cálculo efetuado.
Ano 2020 concluído
Tamanho do dataset pré-filtragem:  2685053
Tamanho do dataset pós-filtragem:  676047
Taxa de filtragem:  0.7482183778122815
Ano carregado.


/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_56635/3476108975.py:8: UserWarning: `meta` is not specified

Cálculo efetuado.
Ano 2021 concluído


In [28]:
contagem_sexo = enem19.TP_SEXO.value_counts().compute()

In [22]:
persist_df(agg, 'anual/total_anos_norm_enem.parquet')

## Estratégia 2

Relação 

ENEM <--> CENSO ESCOLAR

In [7]:
enem = filter_enem(enem)

Tamanho do dataset pré-filtragem:  5095171
Tamanho do dataset pós-filtragem:  1033757
Taxa de filtragem:  0.7971104404543047


In [8]:
enem.head()

,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q012,Q019,Q022,Q024,Q025
26,2019,2,F,1,1,3,4101408,Apucarana,PR,4.0,1.0,1.0,618.2,744.7,636.3,713.7,1.0,160.0,200.0,200.0,180.0,160.0,900.0,E,E,B,B,2,E,A,B,C,B,B,B,C,B,B
28,2019,3,F,1,1,2,4300604,Alvorada,RS,2.0,1.0,1.0,430.4,466.8,515.9,394.1,1.0,160.0,140.0,120.0,120.0,40.0,580.0,E,B,B,B,3,A,A,B,D,A,B,B,D,B,A
31,2019,2,F,1,2,2,3530607,Mogi das Cruzes,SP,2.0,1.0,1.0,371.8,501.4,518.6,442.0,1.0,140.0,120.0,120.0,140.0,140.0,660.0,E,F,B,D,3,B,A,B,C,A,B,A,D,A,B
34,2019,2,M,1,2,2,2919207,Lauro de Freitas,BA,2.0,1.0,1.0,567.3,584.1,595.8,704.5,1.0,140.0,120.0,140.0,120.0,100.0,620.0,E,C,B,B,2,B,A,B,C,A,B,B,C,A,B
36,2019,2,F,1,1,2,3549102,São João da Boa Vista,SP,2.0,1.0,1.0,NaN,457.9,500.1,NaN,1.0,120.0,120.0,120.0,120.0,80.0,560.0,E,E,C,B,2,D,A,B,C,A,B,B,C,B,B


In [14]:
index_cols = {
    'agg_depend_loc':
        ['NU_ANO','SG_UF_ESC','CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','TP_ESCOLA','TP_DEPENDENCIA_ADM_ESC','TP_LOCALIZACAO_ESC'],
    'agg_depend':
        ['NU_ANO','SG_UF_ESC','CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','TP_ESCOLA','TP_DEPENDENCIA_ADM_ESC'],
    'agg_pub_pvd':
        ['NU_ANO','SG_UF_ESC','CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','TP_ESCOLA'],

    }
agg_funs = {
    'percent': ['TP_SEXO','TP_COR_RACA','TP_ESTADO_CIVIL','TP_STATUS_REDACAO','Q001','Q002','Q003','Q004','Q005','Q006','Q007','Q008','Q009','Q010','Q012','Q019','Q022','Q024','Q025'],
    'median': {
        'TP_FAIXA_ETARIA':'median',
        'NU_NOTA_CN':'median',
        'NU_NOTA_CH':'median',
        'NU_NOTA_LC':'median',
        'NU_NOTA_MT':'median',
        'TP_STATUS_REDACAO':'median',
        'NU_NOTA_COMP1':'median',
        'NU_NOTA_COMP2':'median',
        'NU_NOTA_COMP3':'median',
        'NU_NOTA_COMP4':'median',
        'NU_NOTA_COMP5':'median',
        'NU_NOTA_REDACAO':'median'
    },
    'mean': {
        'NU_NOTA_CN':'mean',
        'NU_NOTA_CH':'mean',
        'NU_NOTA_LC':'mean',
        'NU_NOTA_MT':'mean',
        'TP_STATUS_REDACAO':'mean',
        'NU_NOTA_COMP1':'mean',
        'NU_NOTA_COMP2':'mean',
        'NU_NOTA_COMP3':'mean',
        'NU_NOTA_COMP4':'mean',
        'NU_NOTA_COMP5':'mean',
        'NU_NOTA_REDACAO':'mean'
    }
}

In [20]:
median_pub_pvd, mean_pub_pvd = agg_enem(enem, index_cols['agg_pub_pvd'], agg_funs)

/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not

Percentual calculado
Mediana calculada
Média calculada


In [32]:
median_depend, mean_depend = agg_enem(enem, index_cols['agg_depend'], agg_funs)

/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not

Percentual calculado
Mediana calculada
Média calculada


In [35]:
median_depend_loc, mean_depend_loc = agg_enem(enem, index_cols['agg_depend_loc'], agg_funs)

/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  series.append(df.groupby(index_cols)[col].apply(lambda x: x.value_counts(normalize=True)).compute().unstack(fill_value=0).add_prefix(f'{col}_'))
/tmp/ipykernel_65742/4061576627.py:7: UserWarning: `meta` is not

Percentual calculado
Mediana calculada
Média calculada


In [29]:
persist_df(median_pub_pvd, 'pub_pvd_mediana.parquet')
persist_df(mean_pub_pvd, 'pub_pvd_media.parquet')

In [33]:
persist_df(median_depend, 'depend_mediana.parquet')
persist_df(mean_depend, 'depend_media.parquet')

In [36]:
persist_df(median_depend_loc, 'depend_loc_mediana.parquet')
persist_df(mean_depend_loc, 'depend_loc_media.parquet')